In [1]:
import copy
import random as rd
import csv
import math

In [2]:
walking_speed = 1.4  # m/s

routeTable = {}
roadLengthTable = {}
arrivalTime = {}
arrivalProfile = {}

#Route table initialization
with open('roadCalcs_routeTable.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        profile = row[0]
        route = list(filter(lambda x:x!='',row[1:]))
        routeTable[profile] = route


#Route length initialization
with open('roadCalcs_roadLengthTable.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        profile = row[0]
        length = float(row[1])
        roadLengthTable[profile] = {
            'length': length,
            'walking_time': round(length / walking_speed) # Calculate walking time in seconds
        }

#population and arrival initialization
with open('roadCalcs_arrivalTime.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        profile = row[0]
        time = float(row[2])
        population = int(row[1])
        arrivalTime[profile] = {
            'population': population,
            'time': time
        }

#arrival profile
with open('roadCalcs_arrivalProfile.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    data = list(reader)
    headers = data[0]
    arrivalData = data[1:]

    for row in arrivalData:
        profile = row[0]
        timeProfile = row[1:]
        arrivalProfile[profile] = {}
        for idx,pct in enumerate(timeProfile):
            arrivalProfile[profile][(idx*3600)] = 0 if pct == '' else abs(float((pct)))


In [3]:
def round_down(num, divisor):
    return num - (num%divisor)

In [4]:
class person:
    def __init__(self,personIndex,timeAtNode,pathList,nodeDetail):
        self.idx = personIndex
        self.curNode = pathList[0]
        self.curNodeIdx = 0
        self.timeAtCurNode = timeAtNode
        self.nodeList = pathList
        self.node_detail = nodeDetail 
        self.timeAtNextNode = timeAtNode + self.calculateTimeToNextNode()

    def __str__(self):
        return f"{self.idx} {self.curNode} {self.timeAtCurNode} {self.timeAtNextNode}"

    def calculateTimeToNextNode(self):
        # print(self.node_detail)
        timeToSpendAtCurNode = self.node_detail[self.curNode]['walking_time']
        return timeToSpendAtCurNode

    def updateAgentLocation(self,curTime):
        if curTime == self.timeAtNextNode:
            #Reached Last Step, remove details
            if self.curNode == self.nodeList[-1]:
                self.curNode = 'end'
                self.timeAtNextNode = -1
                return
    
            self.curNodeIdx+=1
            self.curNode = self.nodeList[self.curNodeIdx]
            self.timeAtCurNode = curTime
            self.timeAtNextNode = curTime + self.calculateTimeToNextNode()
            return

In [5]:
print(routeTable) #node paths
print(roadLengthTable)
print(arrivalTime)
print(arrivalProfile)

{'Stadium': ['A', 'B', 'C', 'D', 'C'], 'Stadium Staff': ['H', 'A', 'C', 'B'], 'Fortress_Sports Climbing 1': ['C', 'E', 'F', 'B'], 'Fortress Staff_Sports Climbing 1': ['H', 'C', 'D', 'E'], 'Fortress_Sports Climbing 2': ['A', 'B', 'C', 'D', 'E', 'F', 'G'], 'Fortress Staff_Sports Climbing 2': ['H', 'G', 'F', 'E'], 'Indoor Arena': ['D', 'C', 'A', 'F'], 'Indoor Arena Staff': ['F', 'E', 'B', 'A', 'B', 'A']}
{'A': {'length': 10.0, 'walking_time': 7}, 'B': {'length': 20.0, 'walking_time': 14}, 'C': {'length': 30.0, 'walking_time': 21}, 'D': {'length': 40.0, 'walking_time': 29}, 'E': {'length': 5.0, 'walking_time': 4}, 'F': {'length': 15.0, 'walking_time': 11}, 'G': {'length': 25.0, 'walking_time': 18}, 'H': {'length': 33.0, 'walking_time': 24}}
{'Stadium': {'population': 45000, 'time': 600.0}, 'Stadium Staff': {'population': 4500, 'time': 900.0}, 'Fortress_Sports Climbing 1': {'population': 4500, 'time': 120.0}, 'Fortress Staff_Sports Climbing 1': {'population': 450, 'time': 300.0}, 'Fortress_

In [6]:
#simulate arrival time of people coming in a lift and moving through from one to the next and calculate occupancy of each node over time
simulationTime = 1800 #seconds
#Escalators
peoplePerHour = 1500


#keep track of time when people arrive at a node, where they are and when they should leave.

arrivalRatePerSecond = peoplePerHour / 3600
numberOfPeopleGenerated = {}
numberOfPeople = {}
peopleList = []
nodeOccupancies = {}

for profile in (routeTable):
     numberOfPeopleGenerated[profile] = 0


for curTime in range(simulationTime):
    timeBucket = round_down(curTime, 3600)
    if curTime not in nodeOccupancies:
        nodeOccupancies[curTime] = {}
        nodeOccupancies[curTime]['end'] = 0
        for key in roadLengthTable:
            nodeOccupancies[curTime][key] = 0

#loop through profiles and get the number of people
    for profile in arrivalProfile:
        numberOfArrivals = int(arrivalProfile[profile][timeBucket] * arrivalTime[profile]['population'])
        if curTime < timeBucket+arrivalTime[profile]['time']:
            rateOfPeople = (numberOfArrivals/arrivalTime[profile]['time'])
            numberOfPeopleGenerated[profile] += rateOfPeople

            profileNodePaths = routeTable[profile]
            nodeDetails = roadLengthTable
            personArrivalAmount = math.floor(numberOfPeopleGenerated[profile])
    ############################################################################################################
            ## Lift Arrival Methodd
            # offset = idx * 12
            # if (curTime+offset)%nodeRoundTime[paths] == 0:# People Arrive at node 1
            #     timeBucket = round_down(curTime, 3600)
            #     liftOccupancy = int(pathLiftCapacity[paths] * liftCapacityArrivalRate[timeBucket])
            #     for personIndex in range(liftOccupancy):
            #         peopleList.append(person(len(peopleList),curTime,nodeList,nodeDetails))
    ##############################################################################################################        

            for personIndex in range(int(personArrivalAmount)):
                peopleList.append(person(len(peopleList),curTime,profileNodePaths,nodeDetails))

        numberOfPeopleGenerated[profile]-=math.floor(numberOfPeopleGenerated[profile])

    for agent in peopleList:
        agent.updateAgentLocation(curTime)
        agentLocation = agent.curNode
        nodeOccupancies[curTime][agentLocation] += 1

print(nodeOccupancies)

{0: {'end': 0, 'A': 18, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0, 'G': 0, 'H': 0}, 1: {'end': 0, 'A': 37, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0, 'G': 0, 'H': 0}, 2: {'end': 0, 'A': 56, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0, 'G': 0, 'H': 0}, 3: {'end': 0, 'A': 75, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0, 'G': 0, 'H': 1}, 4: {'end': 0, 'A': 93, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0, 'G': 0, 'H': 1}, 5: {'end': 0, 'A': 112, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0, 'G': 0, 'H': 1}, 6: {'end': 0, 'A': 131, 'B': 0, 'C': 0, 'D': 0, 'E': 0, 'F': 0, 'G': 0, 'H': 2}, 7: {'end': 0, 'A': 132, 'B': 18, 'C': 0, 'D': 0, 'E': 0, 'F': 0, 'G': 0, 'H': 2}, 8: {'end': 0, 'A': 131, 'B': 37, 'C': 0, 'D': 0, 'E': 0, 'F': 0, 'G': 0, 'H': 2}, 9: {'end': 0, 'A': 131, 'B': 56, 'C': 0, 'D': 0, 'E': 0, 'F': 0, 'G': 0, 'H': 2}, 10: {'end': 0, 'A': 131, 'B': 75, 'C': 0, 'D': 0, 'E': 0, 'F': 0, 'G': 0, 'H': 3}, 11: {'end': 0, 'A': 132, 'B': 93, 'C': 0, 'D': 0, 'E': 0, 'F': 0, 'G': 0, 'H': 3}, 12: {'end': 0, 'A': 131, 

In [7]:
# Get headers from one of the inner dictionaries
fieldnames = ['time'] + list(next(iter(nodeOccupancies.values())).keys())

with open('roadCalcs_Output.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for time, node_values in nodeOccupancies.items():
        row = {'time': time}
        row.update(node_values)
        writer.writerow(row)